# Descente de gradient

In [2]:
# make points for regression
from sklearn.datasets import make_regression
x, y = make_regression(n_samples = 100, n_features = 1, n_informative = 1, noise = 10, random_state = 1)

Formula : https://medium.com/analytics-vidhya/vectorized-implementation-of-gradient-descent-in-linear-regression-12a10ea37210

### Python Pur

In [16]:
import numpy as np
   
def main(x, y, alpha, n):
    X = np.array([np.ones(len(x)), x]).T
    y = y.reshape(-1,1)

    am = alpha/len(x)
    theta = np.zeros((2,1))
    
    for _ in range(n):
        theta -= am * X.T.dot((X.dot(theta) - y))

    return theta

In [17]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 10000)

43 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numba

In [27]:
from numba import jit, vectorize, float64, int64
import numpy as np

@jit(float64[:,:](float64[:], float64[:], float64, int64), nopython=True, fastmath=True) # fastmath permet quelques % d'optimisation
def main(x, y, alpha, n):
    X = np.column_stack((np.ones(len(x)), x))
    y = y.copy() # memoire contigue
    y = y.reshape(-1,1)

    am = alpha/len(x)
    
    theta = np.zeros((2,1))
    XT = X.T
    for _ in range(n):
        # .dot more performant than @
        # t1 = X.dot(theta)
        # grad = X.T.dot(t1 - y)
        # theta -= am * grad
        theta -= am * XT.dot(X.dot(theta) - y)

    return theta

### Brouillon

In [24]:
from numba import jit, vectorize, float64, int64
import numpy as np

# @vectorize(['float64(float64, float64, float64, float64)'])
# def v(X, y, am, theta):
#     return am * X.T.dot((X.dot(theta) - y))

# @jit(nopython=True)
@jit(float64[:,:](float64[:], float64[:], float64, int64), nopython=True) # * Le typage m'a pas l'air si utile
def main(x, y, alpha, n):

    X = np.column_stack((np.ones(len(x)), x))
    y = y.copy() # memoire contigue
    y = y.reshape(-1,1)

    am = alpha/len(x)
    
    theta = np.zeros((2,1))
    for _ in range(n):
        # .dot more performant than @
        theta -= am * X.T.dot(X.dot(theta) - y)

    # * J'aimerais utilise @vectorize vu que ça m'a l'air adapté à du SIMD mais bon...
    # theta = np.zeros((2,1), dtype=np.float64)
    # for i in range(n):
    #     theta = theta - v(X, y, am, theta) 

    return theta

In [28]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 1000)

1.25 ms ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Cython

In [2]:
%load_ext Cython

In [42]:
%%cython
import cython
import numpy as np
cimport numpy as cnp

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.binding(False)
cpdef main(cnp.ndarray[cnp.float64_t] x, cnp.ndarray[cnp.float64_t] y, float alpha, int n):
    cdef cnp.ndarray[cnp.float64_t, ndim=2] X = np.column_stack((np.ones(len(x)), x))
    cdef cnp.ndarray[cnp.float64_t, ndim=2] Y  = y.reshape(-1, 1)

    cdef float am = alpha / len(x)
    cdef cnp.ndarray[cnp.float64_t, ndim=2] theta = np.zeros((2,1))

    cdef cnp.ndarray[cnp.float64_t, ndim=2] t1, grad
    for _ in range(n):
        # ! Je comprends rien avec .dot, np.dot(), à la mano, ... rien n'est optimisé
        # ! https://imsc.uni-graz.at/haasegu/Lectures/GPU_CUDA/Lit/seljebotn_cython.pdf (peut-être une implémentation de dot)
        # ! Enlever la transposé aussi mais c'est le dot qui consomme tout
        # t1 = X.dot(theta)
        # grad = X.T.dot(t1 - Y)
        # theta -= am * grad

        theta -= am * X.T.dot(X.dot(theta) - Y)
    return theta


In [39]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 10000)

21.5 ms ± 462 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Brouillon

In [64]:
%%cython
# distutils: language = c++
# cython: boundscheck=False, language_level=3

import cython
import numpy as np
cimport numpy as cnp

cnp.import_array()

# @cython.wraparound(False) # indice négatifs, ici inutile
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.binding(False)
cpdef main(cnp.ndarray[cnp.double_t] x, cnp.ndarray[cnp.double_t] y, float alpha, int n):
    cdef cnp.ndarray[cnp.double_t, ndim=2] X = np.column_stack((np.ones(len(x)), x))
    cdef cnp.ndarray[cnp.double_t, ndim=2] Y  = y.reshape(-1, 1)

    cdef float am = alpha / len(x)
    cdef cnp.ndarray[cnp.double_t, ndim=2] theta = np.zeros((2,1))

    cdef cnp.ndarray[cnp.double_t, ndim=2] t1, grad
    for _ in range(n):
        # .dot more performant than @
        t1 = X.dot(theta)
        grad = X.T.dot(t1 - Y)
        theta -= am * grad

        # theta -= am * X.T.dot(X.dot(theta) - Y)
    return theta

In [65]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 10000)

47.1 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [107]:
%%cython
import cython
import numpy as np
cimport numpy as cnp
from cython cimport boundscheck, wraparound

cdef extern from "cblas.h":
    cnp.double_t cblas_ddot(const int N, const double *X, const int incX, const double *Y, const int incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double dot_product(cnp.ndarray[cnp.double_t, ndim=1] x, cnp.ndarray[cnp.double_t, ndim=1] y):
    cdef int N = x.shape[0]
    cdef int incX = 1
    cdef int incY = 1

    cdef double result = cblas_ddot(N, &x[0], incX, &y[0], incY)
    return result


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.binding(False)
cpdef main(cnp.ndarray[cnp.float64_t, ndim=1] x, cnp.ndarray[cnp.float64_t, ndim=1] y, double alpha, int n):
    cdef cnp.ndarray[cnp.float64_t, ndim=2] X = np.column_stack((np.ones_like(x), x.reshape(-1, 1)))
    cdef cnp.ndarray[cnp.float64_t, ndim=2] Y = y.reshape(-1, 1)

    cdef double am = alpha / len(x)
    cdef cnp.ndarray[cnp.float64_t, ndim=2] theta = np.zeros((2, 1), dtype=np.float64)
    
    # cdef cnp.ndarray[cnp.float64_t, ndim=2] t1, grad

    cdef cnp.ndarray[cnp.float64_t, ndim=2] t1, grad
    cdef int i
    for i in range(n):
        t1 = dot_product(X, theta) 
        grad = dot_product(X.T, t1 - Y)
        theta -= am * grad

    #     theta -= am * X.T.dot(X.dot(theta) - Y)
    return theta


In [108]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 10000)
# main(x, y, 0.01, 1000)

51 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1]:
%%cython
import cython
import numpy as np
cimport numpy as np

# @cython.wraparound(False) # indice négatifs, ici inutile
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
@cython.binding(False)
def main(x, y, double alpha, int n):
    X = np.column_stack((np.ones(len(x)), x))
    Y  = y.reshape(-1, 1)

    cdef double am = alpha / len(x)
    cdef np.ndarray[np.float64_t, ndim=2] theta = np.zeros((2,1))

    for _ in range(n):
        # .dot more performant than @
        t1 = X.dot(theta)
        grad = np.transpose(X).dot(t1 - Y)
        theta -= am * grad

        # theta -= am * X.T.dot(X.dot(theta) - Y)
    return theta

UsageError: Cell magic `%%cython` not found.


In [25]:
x = np.array([-0.61175641, -0.24937038,  0.48851815,  0.76201118,  1.51981682,  0.37756379,  0.51292982, -0.67124613, -1.39649634,  0.31563495, -0.63699565, -0.39675353, -1.10061918,  0.90085595, -1.09989127,  0.82797464, -0.07557171, -0.35224985, -0.67066229, -1.07296862, -0.30620401,  2.18557541,  0.86540763,  0.19829972, -0.38405435, -0.68372786,  0.05080775,  0.58281521,  1.25286816, -0.75439794, -0.34934272, -0.88762896,  0.18656139,  0.87616892,  0.83898341, -0.50446586, -0.34385368,  1.6924546 , -2.3015387 ,  0.93110208,  2.10025514,  1.46210794, -0.84520564, -0.87785842, -0.3224172 ,  0.88514116,  0.16003707,  1.13162939, -0.37528495,  0.50249434, -0.20889423,  0.12015895,  0.58662319,  0.3190391 , -0.69166075,  0.69803203,  1.19891788, -0.20075807,  0.53035547,  0.74204416,  0.41005165,  0.11900865, -0.7612069 ,  0.42349435,  0.30017032, -1.1425182 ,  0.18515642, -0.93576943, -0.62000084, -1.11731035, -1.44411381, -0.22232814,  1.62434536,  0.61720311, -0.6871727 ,  0.07734007, -0.0126646 , -0.63873041,  1.13376944,  1.74481176,  0.90159072, -2.06014071,  0.2344157 , -0.17242821,  0.12182127,  1.14472371, -0.12289023, -0.74715829,  0.28558733, -2.02220122,  0.23009474, -0.26788808, -0.52817175,  1.12948391,  0.19091548, -0.29809284,  1.65980218,  0.04359686,  0.04221375, -0.19183555])
y = np.array([-52.4568854, -15.39439866, 31.09298076, 81.29922354,133.5013812, 31.88110117, 37.20705164,-40.03179671, -106.17554223, 24.639066, -48.50046802,-20.18047019,-92.07348237, 54.55792402,-88.03269964, 67.24781801, -9.89302113,-41.93584528,-59.73487076,-70.52478452, -26.77450907,177.61698746, 67.46572572, 21.33659327,-24.77159853, -48.34153575,2.47827403, 45.80048542,108.55222544,-62.75382452, -27.06345223,-66.26151874, 16.88850749, 72.89287434, 92.45898132, -31.70698544,-42.05759494,158.71803893, -189.16666685, 71.82648428, 167.51735412,111.50394103,-69.32543107,-64.8108726, -33.79908714, 72.34371505, 26.9415416,85.89817092,-29.54171881, 37.39534229, -29.28741926, 12.02641447, 63.3414933,26.43950925,-47.65582687, 61.57186756, 96.14806434, -6.32112365, 21.45691711, 58.19302951, 42.33541142,3.75391925,-44.04667419, 36.96121561, 30.16415401, -83.90759068, 17.32833857,-73.56373515,-34.97208383,-83.44356418, -117.44672502,-23.8844141, 141.09227065, 51.3687163, -57.5068713, 18.48181592,-15.18861638,-48.1692154,95.78274149,145.70060609, 97.38413861, -174.06609658, 11.19686853,-18.44845625, 27.50826761, 95.49878769, -9.59820867,-56.39070754, 30.82136659, -181.59266703,4.52545955,-12.54687448,-38.97063311, 83.28349563, 16.48951451, -24.00962303,133.02472573, 12.06315399, 27.25915157,-20.89459341])

%timeit main(x, y, 0.01, 10000)
# main(x, y, 0.01, 1000)

45 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
